# Regras de associação

Alunos: 

        Leticia Bossatto Marchezi        | RA: 791003
        Luís Augusto Simas do Nascimento | RA: 790828

Instalação e importação da biblioteca `efficient-apriori`.

In [54]:
pip install efficient-apriori

In [55]:
from efficient_apriori import apriori

## Preparação do conjunto de dados

Escolheu-se um conjunto de dados referente a prescrição de lentes de contato para pacientes com base em características e sintomas descritas pelos atributos.

http://archive.ics.uci.edu/ml/datasets/Lenses

O conjunto de dados escolhido não possui *header*, portanto foi necessário rotular os atributos no momento da leitura do arquivo csv.

In [56]:
import pandas as pd 

url='https://drive.google.com/uc?id=1qeumgMhUg1LC4GoWAQynVcOxxlqzWOTV'
df = pd.read_csv(url,header=None,names=["age","prescription","astigmatic", "tearProductionRate","classe"],sep="  ",engine='python')

Após ler o conjunto de dados, o primeiro passo para o tratamento foi a remoção do atributo de classe.

In [57]:
df.drop(columns="classe",inplace=True)
df.head()

,age,prescription,astigmatic,tearProductionRate
1,1,1,1,1
2,1,1,1,2
3,1,1,2,1
4,1,1,2,2
5,1,2,1,1


Verificou-se que o conjunto de dados, apesar de conter apenas atributos nominais, atribui a eles valores numéricos para identificar cada característica possível. Tendo isso em vista, fez-se necessária a conversão dos atributos para uma representação mais semântica da característica descrita, tornando possível a extração de regras que contenham algum significado no contexto do problema.

In [58]:
age = {
    '1': 'young',
    '2': 'pre-presbyopic',
    '3': 'presbyopic'
}

prescription = {
     '1': 'myope',
     '2': 'hypermetrope'
}

astigmatic = {
    '1': 'non astigmatic', 
    '2': 'astigmatic'
}

tearProductionRate = {
    '1': 'tear production rate reduced',
    '2': 'tear production rate normal'
}

df['age'] = list(map(lambda x: age[str(x)], df['age']))
df['prescription'] = list(map(lambda x: prescription[str(x)], df['prescription']))
df['astigmatic'] = list(map(lambda x: astigmatic[str(x)], df['astigmatic']))
df['tearProductionRate'] = list(map(lambda x: tearProductionRate[str(x)], df['tearProductionRate']))
df.head()

,age,prescription,astigmatic,tearProductionRate
1,young,myope,non astigmatic,tear production rate reduced
2,young,myope,non astigmatic,tear production rate normal
3,young,myope,astigmatic,tear production rate reduced
4,young,myope,astigmatic,tear production rate normal
5,young,hypermetrope,non astigmatic,tear production rate reduced


Após a conversão dos atributos, foi possível verificar que todos os atributos do conjunto possuem algum valor, ou seja, não há atributos nulos no conjunto.

In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24 entries, 1 to 24
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   age                 24 non-null     object
 1   prescription        24 non-null     object
 2   astigmatic          24 non-null     object
 3   tearProductionRate  24 non-null     object
dtypes: object(4)
memory usage: 960.0+ bytes


O próximo passo se deu através da conversão do *dataframe* para o formato aceito pelo algoritmo: uma lista de tuplas.

In [60]:
t_list = [tuple(linha) for linha in df.values.tolist()]
t_list

[('young', 'myope', 'non astigmatic', 'tear production rate reduced'),
 ('young', 'myope', 'non astigmatic', 'tear production rate normal'),
 ('young', 'myope', 'astigmatic', 'tear production rate reduced'),
 ('young', 'myope', 'astigmatic', 'tear production rate normal'),
 ('young', 'hypermetrope', 'non astigmatic', 'tear production rate reduced'),
 ('young', 'hypermetrope', 'non astigmatic', 'tear production rate normal'),
 ('young', 'hypermetrope', 'astigmatic', 'tear production rate reduced'),
 ('young', 'hypermetrope', 'astigmatic', 'tear production rate normal'),
 ('pre-presbyopic', 'myope', 'non astigmatic', 'tear production rate reduced'),
 ('pre-presbyopic', 'myope', 'non astigmatic', 'tear production rate normal'),
 ('pre-presbyopic', 'myope', 'astigmatic', 'tear production rate reduced'),
 ('pre-presbyopic', 'myope', 'astigmatic', 'tear production rate normal'),
 ('pre-presbyopic',
  'hypermetrope',
  'non astigmatic',
  'tear production rate reduced'),
 ('pre-presbyopic',
 

## Extração das regras

Com o conjunto de dados devidamente limpo e formatado, basta aplicá-lo ao algoritmo de extração de regras. Para aplicação do algoritmo estabeleceu-se os valores mínimos de suporte e confiança como $0,2$ e $0,5$, respectivamente.

In [61]:
itemsets, rules = apriori(t_list, min_support=0.2, min_confidence = 0.5)

É possível verificar os itemsets gerados.

In [62]:
itemsets

{1: {('astigmatic',): 12,
  ('hypermetrope',): 12,
  ('myope',): 12,
  ('non astigmatic',): 12,
  ('pre-presbyopic',): 8,
  ('presbyopic',): 8,
  ('tear production rate normal',): 12,
  ('tear production rate reduced',): 12,
  ('young',): 8},
 2: {('astigmatic', 'hypermetrope'): 6,
  ('astigmatic', 'myope'): 6,
  ('astigmatic', 'tear production rate normal'): 6,
  ('astigmatic', 'tear production rate reduced'): 6,
  ('hypermetrope', 'non astigmatic'): 6,
  ('hypermetrope', 'tear production rate normal'): 6,
  ('hypermetrope', 'tear production rate reduced'): 6,
  ('myope', 'non astigmatic'): 6,
  ('myope', 'tear production rate normal'): 6,
  ('myope', 'tear production rate reduced'): 6,
  ('non astigmatic', 'tear production rate normal'): 6,
  ('non astigmatic', 'tear production rate reduced'): 6}}

E também todas as regras extraídas, sendo exibidas também suas respectivas métricas de suporte, confiança, lift e convicção.

In [63]:
for rule in rules:
  print(rule)

{hypermetrope} -> {astigmatic} (conf: 0.500, supp: 0.250, lift: 1.000, conv: 1.000)
{astigmatic} -> {hypermetrope} (conf: 0.500, supp: 0.250, lift: 1.000, conv: 1.000)
{myope} -> {astigmatic} (conf: 0.500, supp: 0.250, lift: 1.000, conv: 1.000)
{astigmatic} -> {myope} (conf: 0.500, supp: 0.250, lift: 1.000, conv: 1.000)
{tear production rate normal} -> {astigmatic} (conf: 0.500, supp: 0.250, lift: 1.000, conv: 1.000)
{astigmatic} -> {tear production rate normal} (conf: 0.500, supp: 0.250, lift: 1.000, conv: 1.000)
{tear production rate reduced} -> {astigmatic} (conf: 0.500, supp: 0.250, lift: 1.000, conv: 1.000)
{astigmatic} -> {tear production rate reduced} (conf: 0.500, supp: 0.250, lift: 1.000, conv: 1.000)
{non astigmatic} -> {hypermetrope} (conf: 0.500, supp: 0.250, lift: 1.000, conv: 1.000)
{hypermetrope} -> {non astigmatic} (conf: 0.500, supp: 0.250, lift: 1.000, conv: 1.000)
{tear production rate normal} -> {hypermetrope} (conf: 0.500, supp: 0.250, lift: 1.000, conv: 1.000)
{hy